In [1]:
from tqdm import tqdm_notebook
import os
from time import sleep
import re
import pandas as pd
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from constants import data_directory
import numpy as np
from copy import copy

## Loading pathes from ftp

In [ ]:
def scantree(path):
    """Recursively yield DirEntry objects for given directory."""
    for entry in os.scandir(path):
        if entry.is_dir(follow_symlinks = False):
            yield from scantree(entry.path)
        else:
            yield entry

In [ ]:
video_dir = '/mnt/HARD/MinMax94/data/Video'
video_pathes = '/mnt/HARD/MinMax94/data/video_pathes'

road_list = [d for d in os.listdir(video_dir)
            if os.path.isdir(os.path.join(video_dir, d))]

roads_already_saved = [f.rsplit('_', 1)[0] for f in os.listdir(video_pathes)
                        if os.path.isfile(os.path.join(video_pathes, f))
                        if not f.startswith('.')]

roads_to_save = [road for road in road_list if road not in roads_already_saved]

for road in tqdm_notebook(roads_to_save):
    i = 0
    while (i==0):
        
        try:
            print('Road #{0}: loading started'.format(road))
            file_list_gen = scantree(os.path.join(video_dir, road))
            file_list = [f.path for f in tqdm_notebook(file_list_gen)]

            print('{0}_pathes.txt loading...'.format(road))
            file_to_save = open('/mnt/HARD/MinMax94/data/video_pathes/{0}_pathes.txt'.format(road), 'w')
            i = 1
            for item in file_list:
                file_to_save.write("%s\n" % item)
              
        except:
            print('here')
            sleep(1)

## Parse file names to get date_time

In [4]:
'image([0-9]{2})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})-[0-9]{2}\.(jpg|JPG|jpeg|JPEG)'
#image13-07-31_18-24-17-37.jpg

'image([0-9]{2})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})\.(jpg|JPG|jpeg|JPEG)'
#image13-07-31_18-24-17.jpg

'img_([0-9]{4})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})-[0-9]{2}\.(jpg|JPG|jpeg|JPEG)'
#img_2013-08-05_17-30-43-67.jpg

'([0-9]{2})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})-[0-9]{2}'
#13-08-02_21-28-23-45

'image([0-9]{4})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})-[0-9]{2}\.(jpg|JPG|jpeg|JPEG)'
#image2013-08-25_23-47-00.jpg

'img_([0-9]{4})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})_[0-9]{2}\.(jpg|JPG|jpeg|JPEG)'
#img_2013-08-25_23-54-27_80.jpg

'image\.(jpg|JPG|jpeg|JPEG)'
#image.jpg

'image([0-9]{2})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})-([0-9]{2})\.(jpg|JPG|jpeg|JPEG)'
#image15-05-26_16-09-25-78

'img_-([0-9]{4})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})-([0-9]{2})\.(jpg|JPG|jpeg|JPEG)'
#img_-2015-05-26_15-02-07-00.jpg

'image([0-9]{4})-([0-9]{2})-([0-9]{2})_([0-9]{2})-([0-9]{2})-([0-9]{2})\.(jpg|JPG|jpeg|JPEG)'
#image2013-08-19_23-41-00.jpg

'bspimage_([0-9]{2})([0-9]{2})([0-9]{2})_([0-9]{2})([0-9]{2})([0-9]{2})\.(jpg|JPG|jpeg|JPEG)'
#bspimage_13-07-31_18-24-17.jpg

'Image-(\d{4})(\d{2})(\d{2})-(\d{2})(\d{2})(\d{2})\.(jpg|JPG|jpeg|JPEG)'
#Image-20170831-195708.jpg

'Image-(\\d{4})(\\d{2})(\\d{2})-(\\d{2})(\\d{2})(\\d{2})\\.(jpg|JPG|jpeg|JPEG)'

In [2]:
pathes_list = ['image13-07-31_18-24-17-37.jpg',
              'image13-07-31_18-24-17.jpg',
              'img_2013-08-05_17-30-43-67.jpg',
              '13-08-02_21-28-23-45',
              'image2013-08-25_23-47-00.jpg',
              'img_2013-08-25_23-54-27_80.jpg',
              'image15-05-26_16-09-25-78',
              'img_-2015-05-26_15-02-07-00.jpg',
              'image2013-08-19_23-41-00.jpg',
              'bspimage_13-07-31_18-24-17.jpg',
              'Image-20170831-195708.jpg']

In [5]:
def get_date(path):
    date_time = path.split('/')[-1]
    start = '(^image20)|(^image)|(^img_-20)|(^img_20)|(^bspimage_)|(^Image-20)|(^img20)|(.jpg$)|(.jpeg$)|(.JPG$)|(.JPEG$)'
    date_time = '20' + re.sub(start, '', date_time)
    date_time = ''.join(re.split(r'-|_', date_time))
    try:
        year = int(date_time[:4])
        month = int(date_time[4:6])
        day = int(date_time[6:8])
        hour = int(date_time[8:10])
        minute = int(date_time[10:12])
        second = int(date_time[12:14])
        date_time = pd.Timestamp(year, month, day, hour, minute, second)
    except:
        print(path)
        return None
    
    return date_time

def get_video_dir(path):
    path_split = path.split('/')
    km = path_split[-5]
    road_name = path_split[-6]
    video_dir = '/'.join([road_name, km])
    return video_dir

def get_video_location(path):
    path_split = path.split('/')
    video_location = '/'.join(path_split[-4:])
    return video_location

## Creating csv files with photo information

In [ ]:
video_pathes = '/mnt/HARD/MinMax94/data/video_pathes'
mm94_stations_def = pd.read_csv(data_directory + '/stations_mm94_def.csv')
road_files_list = [os.path.join(video_pathes, f) for f in os.listdir(video_pathes)
                    if os.path.isfile(os.path.join(video_pathes, f))]

stations_saved = [int(f.split('_')[0]) for f in os.listdir(data_directory + '/VIDEO/')
                    if os.path.isfile(os.path.join(data_directory + '/VIDEO/', f))]

for road_file in road_files_list:
    print(road_file)
    df = pd.read_csv(road_file, header=None, names=['full_path'])
    df['date_time'] = df['full_path'].apply(get_date)
    df['video_dir'] = df['full_path'].apply(get_video_dir)
    df['path'] = df['full_path'].apply(get_video_location)
    
    for video_dir in df['video_dir'].unique():
        station_num = mm94_stations_def[mm94_stations_def['video_path']==video_dir]['video_id'].values
        if len(station_num) > 0:
            if station_num[0] not in stations_saved:
                df.loc[df['video_dir']==video_dir, 'video_id'] = station_num[0]
                df_to_file = copy(df[df['video_dir']==video_dir])
                df_to_file = df_to_file.dropna(subset=('video_id', 'date_time'))
                print(video_dir, ' = ' , station_num[0])
                file_csv_path = data_directory + '/VIDEO/' + str(station_num[0]) + '_video.csv'
                df_to_file.to_csv(file_csv_path, index=False)
        else:
            print(video_dir, ' = ??')

In [ ]:
#old_files_list = [os.path.join(video_pathes, f) for f in os.listdir(video_pathes)
#                    if os.path.isfile(os.path.join(video_pathes, f))]
old_files_list = ['/mnt/HARD/MinMax94/data/video_pathes/m18_pathes.txt']

new_files_list = ['/mnt/HARD/MinMax94/data/video_pathes_1/m18_pathes.txt']

bad_words = ['.tmp', '.php', '/1970/', ':']

for old_file, new_file in tqdm_notebook(zip(old_files_list, new_files_list)):
    with open(old_file) as oldfile, open(new_file, 'w') as newfile:
        for line in oldfile:
            if not any(bad_word in line for bad_word in bad_words):
                newfile.write(line)

## Adding date_time_utc for photo information

In [34]:
def utc_time(df, video_id, station_def_path=data_directory + '/stations_mm94_def.csv'):
    station_def = pd.read_csv(station_def_path)
    timezone = station_def['video_timezone'][station_def['video_id'] == video_id].values[0]
    print(video_id, pd.Timedelta(str(timezone) + 'h'))
    df['date_time_utc'] = pd.to_datetime(
        df['date_time'] - pd.Timedelta(str(timezone) + 'h'))
    return df

In [33]:
video_df = utc_time(video_df, video_id)
video_df

0 days 04:00:00


,full_path,date_time,video_dir,path,video_id,date_time_utc
0,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 16:50:00,a113/110km,2013/11/07/img2013-11-07_16-50-00_7797a9.jpg,30119.0,2013-11-07 12:50:00
1,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 17:04:00,a113/110km,2013/11/07/img2013-11-07_17-04-00_0f8f67.jpg,30119.0,2013-11-07 13:04:00
2,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 17:10:00,a113/110km,2013/11/07/img2013-11-07_17-10-00_0e7e7e.jpg,30119.0,2013-11-07 13:10:00
3,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 17:20:00,a113/110km,2013/11/07/img2013-11-07_17-20-00_46a0c7.jpg,30119.0,2013-11-07 13:20:00
4,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 17:34:00,a113/110km,2013/11/07/img2013-11-07_17-34-00_fd35b8.jpg,30119.0,2013-11-07 13:34:00
5,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 17:40:00,a113/110km,2013/11/07/img2013-11-07_17-40-00_e37605.jpg,30119.0,2013-11-07 13:40:00
6,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 17:50:00,a113/110km,2013/11/07/img2013-11-07_17-50-00_c53f8e.jpg,30119.0,2013-11-07 13:50:00
7,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 18:03:00,a113/110km,2013/11/07/img2013-11-07_18-03-00_3725ed.jpg,30119.0,2013-11-07 14:03:00
8,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 18:10:00,a113/110km,2013/11/07/img2013-11-07_18-10-00_56deb1.jpg,30119.0,2013-11-07 14:10:00
9,/mnt/HARD/MinMax94/data/Video/a113/110km/2013/...,2013-11-07 18:20:00,a113/110km,2013/11/07/img2013-11-07_18-20-00_4f4a0b.jpg,30119.0,2013-11-07 14:20:00


In [40]:
video_stations = [int(f.split('_')[0]) for f in os.listdir(data_directory + '/VIDEO/')
                    if os.path.isfile(os.path.join(data_directory + '/VIDEO/', f))]

for video_id in video_stations:
    path = os.path.join(data_directory, 'VIDEO', '{0}_video.csv'.format(video_id))
    video_df = pd.read_csv(path)
    video_df['date_time'] = pd.to_datetime(video_df['date_time'])
    video_df = utc_time(video_df, video_id)
    video_df.to_csv(os.path.join(data_directory, 'VIDEO_1', '{0}_video.csv'.format(video_id)), index=False)

30740 0 days 03:00:00
30697 0 days 04:00:00
30778 0 days 03:00:00
5621 0 days 03:00:00
30119 0 days 04:00:00
5910 0 days 03:00:00
7040 0 days 03:00:00
30614 0 days 03:00:00
7045 0 days 03:00:00
5640 0 days 04:00:00
50789 0 days 04:00:00
5307 0 days 03:00:00
5915 0 days 03:00:00
50398 0 days 03:00:00
30696 0 days 04:00:00
50439 0 days 03:00:00
50414 0 days 03:00:00
50644 0 days 03:00:00
30644 0 days 03:00:00
51235 0 days 03:00:00
5212 0 days 03:00:00
51467 0 days 03:00:00
5419 0 days 03:00:00
51884 0 days 03:00:00
30537 0 days 08:00:00
51627 0 days 03:00:00
50039 0 days 03:00:00
51262 0 days 03:00:00
5504 0 days 03:00:00
30724 0 days 03:00:00
31022 0 days 03:00:00
4124 0 days 03:00:00
50204 0 days 04:00:00
30649 0 days 03:00:00
30232 0 days 04:00:00
869 0 days 03:00:00
50647 0 days 03:00:00
7004 0 days 03:00:00
30106 0 days 04:00:00
5560 0 days 04:00:00
5333 0 days 03:00:00
51461 0 days 03:00:00
6929 0 days 04:00:00
5517 0 days 03:00:00
50279 0 days 03:00:00
30530 0 days 08:00:00
51311 

50374 0 days 04:00:00
30770 0 days 03:00:00
50011 0 days 03:00:00
4120 0 days 03:00:00
50560 0 days 03:00:00
30525 0 days 08:00:00
5595 0 days 03:00:00
51635 0 days 03:00:00
51563 0 days 03:00:00
30335 0 days 04:00:00
50195 0 days 04:00:00
30787 0 days 03:00:00
5424 0 days 03:00:00
51770 0 days 03:00:00
30673 0 days 03:00:00
854 0 days 03:00:00
5707 0 days 03:00:00
52146 0 days 03:00:00
31010 0 days 03:00:00
30209 0 days 04:00:00
5404 0 days 03:00:00
5423 0 days 03:00:00
50671 0 days 03:00:00
30743 0 days 03:00:00
4108 0 days 03:00:00
5919 0 days 03:00:00
4103 0 days 03:00:00
50061 0 days 03:00:00
30551 0 days 08:00:00
5645 0 days 03:00:00
30550 0 days 08:00:00
879 0 days 03:00:00
5733 0 days 03:00:00
50540 0 days 03:00:00
50323 0 days 03:00:00
5611 0 days 03:00:00
5303 0 days 03:00:00
52048 0 days 03:00:00
884 0 days 03:00:00
5623 0 days 03:00:00
30207 0 days 04:00:00
52251 0 days 03:00:00
5988 0 days 03:00:00
5913 0 days 03:00:00
7006 0 days 03:00:00
5925 0 days 03:00:00
51244 0 days

51703 0 days 03:00:00
51848 0 days 03:00:00
7050 0 days 03:00:00
30914 0 days 03:00:00
5648 0 days 04:00:00
5556 0 days 03:00:00
30685 0 days 03:00:00
51166 0 days 03:00:00
31308 0 days 03:00:00
5605 0 days 04:00:00
703 0 days 03:00:00
5742 0 days 03:00:00
7018 0 days 03:00:00
30204 0 days 04:00:00
30765 0 days 03:00:00
51795 0 days 05:00:00
30785 0 days 03:00:00
51712 0 days 06:00:00
51828 0 days 03:00:00
51395 0 days 03:00:00
50763 0 days 03:00:00
51917 0 days 03:00:00
50898 0 days 03:00:00
30714 0 days 03:00:00
30211 0 days 04:00:00
30522 0 days 08:00:00
51882 0 days 03:00:00
6919 0 days 03:00:00
50442 0 days 03:00:00
1897 0 days 03:00:00
30345 0 days 04:00:00
51270 0 days 03:00:00
31211 0 days 03:00:00
5921 0 days 03:00:00
51348 0 days 03:00:00
30229 0 days 04:00:00
30507 0 days 08:00:00
5322 0 days 03:00:00
51279 0 days 03:00:00
50667 0 days 03:00:00
50236 0 days 03:00:00
50851 0 days 03:00:00
5318 0 days 03:00:00
7031 0 days 03:00:00
6938 0 days 03:00:00
30571 0 days 08:00:00
556

5584 0 days 03:00:00
6952 0 days 03:00:00
30620 0 days 03:00:00
5928 0 days 03:00:00
31229 0 days 03:00:00
30708 0 days 03:00:00
30521 0 days 08:00:00
50071 0 days 03:00:00
5604 0 days 03:00:00
50855 0 days 03:00:00
51792 0 days 03:00:00
30604 0 days 03:00:00
30646 0 days 03:00:00
30346 0 days 04:00:00
50059 0 days 03:00:00
5413 0 days 03:00:00
5929 0 days 03:00:00
50638 0 days 03:00:00
30124 0 days 04:00:00
52250 0 days 03:00:00
51726 0 days 03:00:00
50641 0 days 03:00:00
6944 0 days 03:00:00
5230 0 days 04:00:00
30338 0 days 04:00:00
5220 0 days 04:00:00
5554 0 days 03:00:00
7008 0 days 03:00:00
51844 0 days 03:00:00
7010 0 days 03:00:00
678 0 days 03:00:00
5732 0 days 03:00:00
5578 0 days 03:00:00
7012 0 days 03:00:00
4101 0 days 03:00:00
30680 0 days 03:00:00
50805 0 days 03:00:00
6931 0 days 03:00:00
6993 0 days 03:00:00
51720 0 days 03:00:00
50716 0 days 03:00:00
52196 0 days 03:00:00
50669 0 days 03:00:00
30612 0 days 03:00:00
52197 0 days 03:00:00
50081 0 days 03:00:00
5211 0 d